## "Kommissionelle" exam

## c sharp struct memory storage open ended question

### Difference Between Value Type and Reference Type

In C#, the difference between value types and reference types is fundamental to understanding how memory management works and how data is accessed and manipulated. Here are the key differences:

#### 1. Memory Allocation

- **Value Types**:
  - **Stack Allocation**: Value types are typically allocated on the stack. The stack is a region of memory that stores local variables and method call information.
  - **Inline Storage**: When value types are fields within a class, they are stored inline within the memory allocated for the class instance on the heap.

- **Reference Types**:
  - **Heap Allocation**: Reference types are allocated on the heap. The heap is a region of memory used for dynamic memory allocation.
  - **Reference Storage**: When you declare a variable of a reference type, it stores a reference (or pointer) to the actual data on the heap.

#### 2. Type Examples

- **Value Types**:
  - Primitive types: `int`, `float`, `double`, `char`, `bool`
  - Structs: User-defined structures (e.g., `struct Point { public int X; public int Y; }`)
  - Enums: Enumeration types (e.g., `enum Colors { Red, Green, Blue }`)

- **Reference Types**:
  - Classes: User-defined classes (e.g., `class Person { public string Name; public int Age; }`)
  - Arrays: Arrays of any type (e.g., `int[]`, `string[]`)
  - Delegates: Types that reference methods
  - Interface types: Defined with the `interface` keyword

#### 3. Copying Behavior

- **Value Types**:
  - **Copy by Value**: When you assign one value type variable to another, a copy of the value is made. Changes to one variable do not affect the other.
  - Example:
    ```csharp
    int a = 5;
    int b = a; // b is a copy of a
    b = 10; // a is still 5, b is 10
    ```

- **Reference Types**:
  - **Copy by Reference**: When you assign one reference type variable to another, both variables refer to the same object. Changes to one variable affect the other.
  - Example:
    ```csharp
    Person p1 = new Person { Name = "Alice" };
    Person p2 = p1; // p2 references the same object as p1
    p2.Name = "Bob"; // p1.Name is also "Bob"
    ```

#### 4. Default Values

- **Value Types**:
  - Default values are typically zero or equivalent for the type (e.g., `0` for integers, `0.0` for floats, `false` for bools).
  - Example:
    ```csharp
    int a; // default is 0
    float b; // default is 0.0f
    ```

- **Reference Types**:
  - Default value is `null`, indicating that the variable does not refer to any object.
  - Example:
    ```csharp
    Person p; // default is null
    string s; // default is null
    ```

#### 5. Boxing and Unboxing

- **Value Types**:
  - **Boxing**: Converting a value type to a reference type (object). This involves allocating memory on the heap and copying the value.
  - **Unboxing**: Converting a reference type back to a value type.
  - Example:
    ```csharp
    int a = 5;
    object boxed = a; // boxing
    int b = (int)boxed; // unboxing
    ```

- **Reference Types**:
  - Reference types do not require boxing or unboxing since they are already on the heap and referenced by pointers.

### Summary

- **Value Types**: Stored on the stack, directly contain their data, copy by value, typically faster allocation, default values are type-specific zero equivalents.
- **Reference Types**: Stored on the heap, contain references to their data, copy by reference, can be `null`, involve garbage collection, default value is `null`.

Understanding these differences is crucial for writing efficient and effective C# code, especially when dealing with performance-critical applications or large-scale data structures.


### Covariance and Contravariance in C# Generic Data Types, Especially in a List

Covariance and contravariance provide a way to use more derived types than originally specified, or less derived types, in a consistent manner. This is particularly useful with generics and collections in C#. Here's an explanation of these concepts, particularly in the context of lists.

#### Covariance

Covariance allows a method to return a more derived type than that specified by the generic parameter. This is typically applicable in out (output) positions, such as return types.

- **Covariant Interfaces**: In C#, interfaces can be covariant if they use the `out` keyword. The `IEnumerable<T>` interface is a common example.

```csharp
public interface IEnumerable<out T> : IEnumerable
{
    IEnumerator<T> GetEnumerator();
}

class Animal { }
class Dog : Animal { }

IEnumerable<Dog> dogs = new List<Dog>();
IEnumerable<Animal> animals = dogs; // This is allowed due to covariance

```

Contravariance allows a method to accept parameters of a more generic type than that specified by the generic parameter. This is typically applicable in in (input) positions, such as method parameters.

- Contravariant Interfaces: In C#, interfaces can be contravariant if they use the in keyword. The IComparer<T> interface is a common example.

```csharp
public interface IComparer<in T>
{
    int Compare(T x, T y);
}

class Animal { }
class Dog : Animal { }

IComparer<Animal> animalComparer = GetAnimalComparer();
IComparer<Dog> dogComparer = animalComparer; // This is allowed due to contravariance
```

Important here: Allowing `List<Dog>` to be assigned to `List<Animal>` would break type safety since it would allow adding any Animal to a list of Dog.

```csharp
List<Dog> dogs = new List<Dog>();
List<Animal> animals = dogs; // This is not allowed
animals.Add(new Cat()); // This would be problematic as Cat is not a Dog
```

### Summary

- **Covariance**:
  - Enables more derived types.
  - Applicable to output positions.
  - Example: `IEnumerable<out T>`

- **Contravariance**:
  - Enables more generic types.
  - Applicable to input positions.
  - Example: `IComparer<in T>`

- **Invariant**:
  - `List<T>` is invariant.
  - No assignment compatibility between `List<Dog>` and `List<Animal>` due to potential type safety issues.

Understanding covariance and contravariance helps in designing and using generic interfaces and delegates in a type-safe manner, especially when dealing with collections and hierarchy of types.


## c sharp dotnet order of execution and syncrhonisation context looking at the following scenarios

Method1()
{
var x = SyncMethod1();
var y = SyncMethod2();
}

Method2()
{
var x = await AsyncMethod1();
var y = await AsyncMethod2();
}

Method3 ()
{
var x = AsyncMethod1();
var y = AsyncMethod2();
}

### M1

- Order of Execution: This method executes SyncMethod1 first and waits for it to complete before moving on to SyncMethod2. It is a synchronous execution.
- Synchronization Context: Since both methods are synchronous, they execute sequentially on the calling thread, blocking it until they complete.

### M2

- Order of Execution: This method calls AsyncMethod1 and awaits its completion before moving on to AsyncMethod2. This is an asynchronous execution, but it still waits for each method to complete before proceeding.
- Synchronization Context: If this method is called from a UI thread or an ASP.NET request context, the await operator will capture the synchronization context to continue on the original context after the awaited task completes. If it’s called from a non-UI thread (e.g., a background worker), it will continue on the thread pool.

### M3

- Order of Execution: This method calls AsyncMethod1 and AsyncMethod2 without awaiting them. Both methods will start executing asynchronously, and Method3 will continue executing without waiting for either of them to complete. This can lead to concurrent execution of AsyncMethod1 and AsyncMethod2.
- Synchronization Context: Since the method is not awaited, it does not capture the synchronization context, and both asynchronous methods will execute on the thread pool or whatever context they are running on. The calling thread does not wait for the completion of these asynchronous methods.

## ADO.NET

The rest of the exam was ADO.NET databases functionality with a focus on application design for layer separation. There was a domain class given where exactly database functionality and domain are mingled and this would be the basis for formulating a better design, something like:

```csharp
public class Person {
    public string Name {get; set; }
    public int Age {get; set; }
    public UpdateAge(int age);
    public FindByName(string name);
}
```

The rest was about Provider Classes in ADO.NET and how factories are used to provide provider-independent classes. `SqlDBProvider` is an abastract class: would need a provider-specific implementation like the one for MySQL (cannot do new() on an abstract class). Providing DBFactory gives me the needed provider classes via methods.

#### Provider Classes in ADO.NET

ADO.NET uses provider classes to interact with different databases in a uniform manner. Each provider class is specific to a database type, such as SQL Server, MySQL, Oracle, etc. These classes include:

- `SqlConnection`
- `SqlCommand`
- `SqlDataAdapter`
- `SqlDataReader`

For MySQL, similar classes exist like `MySqlConnection`, `MySqlCommand`, etc. These classes are part of the MySQL.Data namespace.

#### Abstract Classes and Provider-Specific Implementations

The statement mentions `SqlDBProvider` as an abstract class. In object-oriented programming, an abstract class serves as a base class and cannot be instantiated directly. It defines a template for derived classes, ensuring they implement specific methods or properties.

```csharp
public abstract class SqlDBProvider
{
    public abstract void Connect();
    public abstract void ExecuteCommand(string commandText);
}
```

An abstract class in ADO.NET would require a concrete implementation for each specific database provider. For instance:

```csharp
public class MySQLDBProvider : SqlDBProvider
{
    public override void Connect()
    {
        // Implementation for MySQL connection
    }

    public override void ExecuteCommand(string commandText)
    {
        // Implementation for executing a command in MySQL
    }
}
```

As noted, you cannot instantiate an abstract class directly:

```csharp
// This will cause a compile-time error
SqlDBProvider provider = new SqlDBProvider();
```

Instead, you instantiate a derived class:

```csharp
SqlDBProvider provider = new MySQLDBProvider();
```

#### Using Factories for Provider-Independent Classes

Factories in ADO.NET are used to create instances of provider-specific classes in a provider-independent manner. The DbProviderFactory class allows for creating objects for different data sources without hardcoding the provider-specific classes. This enables writing generic data access code that can work with any database.

Example of using DbProviderFactory:

```csharp
DbProviderFactory factory = DbProviderFactories.GetFactory("MySql.Data.MySqlClient");
using (DbConnection connection = factory.CreateConnection())
{
    connection.ConnectionString = "your_connection_string";
    connection.Open();
    
    using (DbCommand command = connection.CreateCommand())
    {
        command.CommandText = "SELECT * FROM Users";
        using (DbDataReader reader = command.ExecuteReader())
        {
            while (reader.Read())
            {
                Console.WriteLine(reader["Username"]);
            }
        }
    }
}

```

In this example, DbProviderFactory is used to create instances of DbConnection, DbCommand, and DbDataReader, allowing the same code to work with different databases by changing the provider.

#### 
Summary
Provider Classes: Specific to each database type, enabling interaction with the database.
Abstract Classes: Define a template for provider classes; cannot be instantiated directly.
Concrete Implementations: Required for each database provider, implementing the abstract class methods.
Factories: Enable provider-independent code by creating instances of provider-specific classes, promoting flexibility and reusability.

Understanding these concepts is crucial for effective database interaction and designing scalable, maintainable data access layers in ADO.NET.